# Visualization

Nov 10th / Yuqi


delays.tbss (TBS)
delays.mcss (MCS)
delays.segments (number of segmentations)

ideal causal relationship: TBS --> number of segements <-- MCS


csv_file
panda_frame

## 1 Import

In [277]:
from plot_helpers import *
from data_helpers import *
from decomp import *

import os, sys, gzip, json
from pathlib import Path
from sortedcontainers import SortedList, SortedDict
from loguru import logger
import pandas as pd
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from IPython.display import JSON
import ijson
import seaborn as sns
import pandas as pd

%load_ext autoreload
%autoreload 2

sns.set_theme(style='darkgrid')  # Options include 'darkgrid', 'whitegrid', 'dark', 'white', and 'ticks'
sns.set()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [278]:
# Remove default handler
logger.remove()
# Add a new handler with level WARNING
logger.add(sys.stdout, level="ERROR")

8

## 2 Initalize paths

In [279]:
DATASETS_DIR = "./data/"
PLOTS_DIR = "./plots/"
JSON_PATH = "jsons/"
IF_SHOW_USAGE = False

## 3 ```class Meas``` for visualizing single dataset

This block aims to construct a clear and consice way of visualization by code encapsulation.


In [280]:
SKIP_FIRST = 1000
SKIP_LAST = 100
class Meas: 
    class Data:
        def __init__(self, datasets_dir,meas_label):
            """
            Meas.Data: Store the data from one measurement
            """
            self.meas_label = meas_label
            self.data_path = datasets_dir + meas_label
            jsons_path = os.path.join(self.data_path, JSON_PATH)
            self.packets = read_json(os.path.join(jsons_path, "packets.json"))
            self.sr_tx_arr = read_json(os.path.join(jsons_path, "sr_tx.json"))
            self.bsr_tx_arr = read_json(os.path.join(jsons_path, "bsr_tx.json"))
            self.bsrupd_arr = read_json(os.path.join(jsons_path, "bsr_upd.json"))
            self.bsrupd_sorted_dict = SortedDict(
                {bsrupd["timestamp"]: bsrupd for bsrupd in self.bsrupd_arr}
            )
            self.sr_bsr_tx_sorted_list = SortedList(
                [sr_tx["timestamp"] for sr_tx in self.sr_tx_arr]
                + [bsr_tx["timestamp"] for bsr_tx in self.bsr_tx_arr]
            )
            self.sched_arr = read_json(os.path.join(jsons_path, "sched.json"))
            self.sched_decid_sorted_dict = SortedDict(
                {sched["decision_ts"]: sched for sched in self.sched_arr}
            )
            self.sched_sched_sorted_dict = SortedDict(
                {sched["schedule_ts"]: sched for sched in self.sched_arr}
            )
            self.packets_rnti_set = set(  # Radio Network Temporary Identifiers
                [
                    item["rlc.attempts"][0]["rnti"]
                    for item in self.packets
                    if item["rlc.attempts"][0]["rnti"] != None
                ]
            )
            print(f"RNTIs in packets of {self.meas_label}: {list(self.packets_rnti_set)}")
            self.mcs_arr_all = read_json(os.path.join(jsons_path, "mcs.json"))
            self.tb_arr = read_json(os.path.join(jsons_path, "tb.json"))
            self.set_rnti = set([item["rnti"] for item in self.mcs_arr_all])
            # filter entries with rnti list(packets_rnti_set)[0]
            if list(self.packets_rnti_set)[0] != None:
                self.mcs_arr = [
                    mcs
                    for mcs in self.mcs_arr_all
                    if mcs["rnti"] == list(self.packets_rnti_set)[0]
                ]
            else:
                self.mcs_arr = self.mcs_arr_all
            self.mcs_sorted_dict = SortedDict(
                {mcs["timestamp"]: mcs for mcs in self.mcs_arr}
            )

    class Delays:
        def __init__(self, data):
            """
            Meas.Delays: Calculate and store delay components, utilizing decomp.py
            """
            self.idt = np.array(
                list(
                    {
                        data.packets[ind]["id"]: data.packets[ind]["ip.in_t"]
                        - data.packets[ind - 1]["ip.in_t"]
                        for ind in range(1, len(data.packets))
                        if data.packets[ind]["ip.in_t"] != None
                        and data.packets[ind - 1]["ip.in_t"] != None
                    }.values()
                )
            )
            self.frame_alignment_delays = np.array(
                list(
                    {
                        packet["id"]: get_frame_alignment_delay(
                            packet,
                            data.sr_bsr_tx_sorted_list,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        for packet in data.packets
                        if get_frame_alignment_delay(
                            packet,
                            data.sr_bsr_tx_sorted_list,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        != None
                        and get_tx_delay(packet) != None
                    }.values()
                )
            )
            self.scheduling_delays = np.array(
                list(
                    {
                        packet["id"]: get_scheduling_delay(
                            packet,
                            data.sched_decid_sorted_dict,
                            data.sched_sched_sorted_dict,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        for packet in data.packets
                        if get_scheduling_delay(
                            packet,
                            data.sched_decid_sorted_dict,
                            data.sched_sched_sorted_dict,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        != None
                        and get_tx_delay(packet) != None
                    }.values()
                )
            )
            self.ran_delays = np.array(
                list(
                    {
                        packet["id"]: get_ran_delay(packet)
                        for packet in data.packets
                        if get_ran_delay(packet) != None
                    }.values()
                )
            )
            self.ran_delays_wo_frame_alignment_delay = np.array(
                list(
                    {
                        packet["id"]: get_ran_delay_wo_frame_alignment_delay(
                            packet,
                            data.sr_bsr_tx_sorted_list,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        for packet in data.packets
                        if get_ran_delay_wo_frame_alignment_delay(
                            packet,
                            data.sr_bsr_tx_sorted_list,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        != None
                    }.values()
                )
            )
            self.ran_delays_wo_scheduling_delay = np.array(
                list(
                    {
                        packet["id"]: get_ran_delay_wo_scheduling_delay(
                            packet,
                            data.sched_decid_sorted_dict,
                            data.sched_sched_sorted_dict,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        for packet in data.packets
                        if get_ran_delay_wo_scheduling_delay(
                            packet,
                            data.sched_decid_sorted_dict,
                            data.sched_sched_sorted_dict,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        != None
                    }.values()
                )
            )
            self.queueing_delays = np.array(
                list(
                    {
                        packet["id"]: get_queueing_delay(packet)
                        for packet in data.packets
                        if get_queueing_delay(packet) != None and get_tx_delay(packet) !=None
                    }.values()
                )
            )
            self.queueing_delays_wo_scheduling_delay = np.array(
                list(
                    {
                        packet["id"]: get_queueing_delay_wo_scheduling_delay(
                            packet,
                            data.sched_decid_sorted_dict,
                            data.sched_sched_sorted_dict,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        for packet in data.packets
                        if get_queueing_delay_wo_scheduling_delay(
                            packet,
                            data.sched_decid_sorted_dict,
                            data.sched_sched_sorted_dict,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        != None
                        and get_tx_delay(packet) != None
                    }.values()
                )
            )
            self.segmentation_delay = np.array(
                list(
                    {
                        packet["id"]: get_segmentation_delay(packet)
                        for packet in data.packets
                        if get_segmentation_delay(packet) != None
                    }.values()
                )
            )
            self.segmentation_delays_wo_scheduling_delay = np.array(
                list(
                    {
                        packet["id"]: get_segmentation_delay_wo_scheduling_delay(
                            packet,
                            data.sched_decid_sorted_dict,
                            data.sched_sched_sorted_dict,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        for packet in data.packets
                        if get_segmentation_delay_wo_scheduling_delay(
                            packet,
                            data.sched_decid_sorted_dict,
                            data.sched_sched_sorted_dict,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        != None
                    }.values()
                )
            )
            self.segments = np.array(
                list(
                    {
                        packet["id"]: get_segments(packet)
                        for packet in data.packets
                        if get_segments(packet) != None and get_tx_delay(packet) !=None
                    }.values()
                )
            )
            self.retx_delays = np.array(
                list(
                    {
                        packet["id"]: get_retx_delay(packet)
                        for packet in data.packets
                        if get_retx_delay(packet) != None 
                    }.values()
                )
            )
            self.mcss=np.array(
                list(
                    {
                        packet["id"]: get_mcs(
                            packet,
                            data.mcs_sorted_dict,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        for packet in data.packets
                        if get_mcs(
                            packet,
                            data.mcs_sorted_dict,
                            slots_per_frame=20,
                            slots_duration_ms=0.5,
                        )
                        != None and get_tx_delay(packet) != None
                    }.values()
                )
            )
            # self.tbss = np.array(
            #     list(
            #         {
            #             packet["id"]: get_tbs(
            #                 packet, data.sched_sorted_dict, slots_duration_ms=0.5
            #             )
            #             for packet in data.packets
            #             if get_tbs(
            #                 packet, data.sched_sorted_dict, slots_duration_ms=0.5
            #             )
            #             != None
            #         }.values()
            #     )
            # )

            # Here, if "tbs" not exist, we assume the the tbs for this packet remains unchanged
            # last_valid_tbs = None  # Initialize to keep track of the last valid TBS value
            # self.tbss = np.array(
            #     [
            #         # Iterate over each packet to fetch TBS or use last valid TBS if None
            #         (
            #             (
            #                 last_valid_tbs := get_tbs(
            #                     packet,
            #                     data.sched_decid_sorted_dict,
            #                     data.sched_sched_sorted_dict,
            #                     slots_duration_ms=0.5,
            #                 )
            #             )
            #             if (
            #                 tbs := get_tbs(
            #                     packet,
            #                     data.sched_decid_sorted_dict,
            #                     data.sched_sched_sorted_dict,
            #                     slots_duration_ms=0.5,
            #                 )
            #             )
            #             is not None and get_tx_delay(packet) != None
            #             else last_valid_tbs
            #                  if get_tx_delay(packet) != None
            #                  else None
            #         )
            #         for packet in data.packets
            #     ]
            # )
            self.tbss = []
            last_valid_tbs = None  # Initialize last_valid_tbs outside the loop

            for packet in data.packets:
                tbs = get_tbs(
                    packet,
                    data.sched_decid_sorted_dict,
                    data.sched_sched_sorted_dict,
                    slots_duration_ms=0.5,
                )
                tx_delay = get_tx_delay(packet)

                if tbs is not None and tx_delay is not None:
                    last_valid_tbs = tbs
                    self.tbss.append(last_valid_tbs)
                elif tx_delay is not None:
                    self.tbss.append(last_valid_tbs)
                else:
                    self.tbss.append(None)
            self.tbss = np.array(
                [
                    item
                    for item in self.tbss
                    if item != None
                ]
            )

            self.packet_size = np.array(
                list(
                    {
                        packet["id"]: get_packet_size(packet)
                        for packet in data.packets
                        if get_packet_size(packet) != None
                        and get_tx_delay(packet) != None
                    }.values()
                )
            )
            self.timestamps = np.array(
                list(
                    {
                        packet["id"]: packet["ip.in_t"]
                        for packet in data.packets
                        if packet["ip.in_t"] != None and get_tx_delay(packet) != None
                    }
                )
            )

    def __init__(self, datasets_dir=DATASETS_DIR, meas_label="s49"):
        """
        Meas: 
            a class which store and analyze 1 group(1 folder) of measurement. The init function utilizes data_helps.py
        Parameters:
            datasets_dir(str): Path of all datasets;
            meas_label(str): The measurement label;
        """
        self.meas_label=meas_label
        self.data=self.Data(datasets_dir, meas_label)
        self.delays=self.Delays(self.data)
        if not os.path.exists(PLOTS_DIR + self.data.meas_label):
            os.makedirs(PLOTS_DIR + self.data.meas_label)

    def checkData(self, name, pkt_idx=0):
        """
        checkData: check the 1st json object of given attribute name

        Parameters:
            name(str): the attribute name of Meas.Delay\
            pkt_idx(int): the index of checked json object
        """
        attr=getattr(self.data, name, None)
        if attr is not None:
            for idx,attr_item in enumerate(attr):
                if idx <pkt_idx:
                    continue
                elif idx == pkt_idx:
                    print(json.dumps(attr_item, indent=4, allow_nan=True))
                else:
                    break
        else:
            print(f"No attribute {name} found in Meas_{self.data.meas_label}.data")

    def listDataAttr(self):
        return list(vars(self.data).keys()) # ['attr1', 'attr2']

    def listDelaysAttr(self):
        return list(vars(self.delays).keys())

    def plotCCDF(self, curve_name,skip_first=SKIP_FIRST, skip_last=SKIP_LAST, figsize=(8,5), plt_show=True, ax_external=None, outlier=35, x_lim=30):
        """
        Meas.plotCCDF:
            plot 1 ccdf among 13 kinds of delay measurement

        Parameters:
            curve_name(str):the delay measurement you want to plot
            skip_first(int): skip first a few packets
            skip_last(int):  skip last a few packets
            figsize(tuple):  figsize
            plt_show(bool):  if or not show and save figure.
            ax_external(object): external plot object
        """
        curve=getattr(self.delays, curve_name, None)
        if curve is not None:
            if curve_name == "packet_size":
                outlier = None
                x_lim=512
            plot_ccdf(curve[skip_first:-skip_last], f"{self.data.meas_label}_{curve_name}", figsize=figsize, ax=ax_external, outlier=outlier, x_lim = x_lim)
            if plt_show:
                plt.tight_layout()
                plt.savefig(f"{PLOTS_DIR}{self.data.meas_label}/{curve_name}_ccdf_plot.png", dpi=300, bbox_inches='tight')
                plt.show()
        else:
            print(f"No attribute {curve_name} found in meas.delays of {self.data.meas_label}.")

    def plotAllCCDF(self, figsize=(24,24),subplot_division=[1,1]):
        """
        Meas.plotAllCCDF:
            plot ccdf for 13 kinds of delay measurement

        Parameters:
            figsize(tuple):  figsize
            subplot_division(list=[x,y]): x-row y-column subplots. If [1,1], figures will be plotted separately.
        """
        curve_names = self.listDelaysAttr()
        curve_names = [curve_name for curve_name in curve_names if curve_name != 'timestamps']
        figure_num=0
        for i in range(0,len(curve_names)):
            if i % (subplot_division[0]*subplot_division[1]) == 0:
                if i != 0:
                    plt.tight_layout()
                    plt.savefig(f"{PLOTS_DIR}{self.data.meas_label}/all_ccdf_plots_{figure_num}.png", dpi=300)
                    figure_num=figure_num+1
                    plt.show()
                _, axs = plt.subplots(subplot_division[0], subplot_division[1], figsize=figsize)

            if subplot_division[0]*subplot_division[1]==1:
                self.plotCCDF(curve_names[i], plt_show=False, ax_external=axs)
            elif subplot_division[0]==1 or subplot_division[1]==1:
                self.plotCCDF(curve_names[i], plt_show=False, ax_external=axs[i % (subplot_division[1]*subplot_division[0])])
            else:
                self.plotCCDF(
                    curve_names[i],
                    plt_show=False,
                    ax_external=axs[
                        (i % (subplot_division[1]*subplot_division[0] ) // subplot_division[1]),
                        i % subplot_division[1],
                    ],
                )
        plt.tight_layout()
        plt.savefig(f"{PLOTS_DIR}{self.data.meas_label}/all_ccdf_plots_{figure_num}.png", dpi=300, bbox_inches="tight")
        plt.show()

    def plotTimeSeries(self, curve_names, curves=None, start_idx=5000, end_idx=6000, figsize=(12,5), marker="*", title="timeseries",plt_show=True, ax_external=None):
        if ax_external is not None:
            ax = ax_external
        else:
            _, ax = plt.subplots(figsize=figsize)
        index_range = slice(start_idx, end_idx)
        for idx,curve_name in enumerate(curve_names):
            if curves is not None:
                ax.plot(curves[idx][index_range], marker=marker, label=f"{self.data.meas_label}: {curve_name}")
            else:
                curve = getattr(self.delays, curve_name, None)
                if curve is not None:
                    ax.plot(curve[index_range], marker=marker, label=f"{self.data.meas_label}: {curve_name}")
                else:
                    print(
                        f"Curve {curve_name} not found in meas.delays of {self.data.meas_label}."
                    )
        ax.set_xlabel("Index")
        ax.set_ylabel("Delay [ms]")
        ax.grid(True,"minor")
        ax.legend()
        if plt_show == True:
            i = 1
            while os.path.exists(os.path.join(PLOTS_DIR, self.data.meas_label, f"{title}_{i}.png")):
                i += 1
            plt.tight_layout()
            plt.savefig(
                f"{PLOTS_DIR}{self.data.meas_label}/{title}_{i}.png", dpi=300, bbox_inches="tight"
            )
            plt.show()

    def dataFrame(self, curve_names, curve_labels=None, csv_path=None, if_save=True):
        """
        Export dataframe of given delay components with given labels, and save it to csv file.

        curve_labels(list of str): data labels
        curve_names(list of str): attributes names of Meas.Delay (use Meas.listDelaysAttr() to check)
        csv_path(str): Optional. default: the data folder of the meas.
        """
        data=dict()
        for idx, curve_name in enumerate(curve_names):
            curve = getattr(self.delays, curve_name, None)
            if curve is not None:
                if curve_labels is not None:
                    data[curve_labels[idx]]=curve
                else:
                    data[curve_names[idx]] = curve

        # Create a pandas DataFrame with each array as a column
        df = pd.DataFrame(
            data
        )
        # Display the first few rows of the DataFrame
        print(df.head())

        # Export the DataFrame to a CSV file
        if csv_path == None:
            csv_path=f"{DATASETS_DIR}{self.data.meas_label}"
            print(csv_path)

        if if_save ==True:
            df.to_csv(os.path.join(csv_path, f"{"_".join(curve_labels)}.csv"), index=True)
            print(f"Dataframe saved to {os.path.join(csv_path, f"{"_".join(curve_labels)}.csv")}")

        return df

    def plotCrossCorrelation(self,  curve_names, curve_labels= None):
        """
        Calculates and returns the Pearson correlation coefficient between two vectors.

        curve_names(list of str): attributes names of Meas.Delay (use Meas.listDelaysAttr() to check)
        curve_labels(list of str): Optional,data labels
        """
        df = self.dataFrame(curve_names, curve_labels=curve_labels, if_save=False)

        fig = plt.figure()
        ax = fig.add_subplot(111)
        correlation_matrix = df.corr()
        sns.heatmap(df.corr(), ax=ax, annot=True)
        plt.show()

        return correlation_matrix

    def plotAutoCorrelation(self, curve_name, label=None, figsize=(8,5), plt_show=True, ax_external=None, outlier = 35):
        """
        Meas.plotCCDF:
            plot 1 autocorr among all kinds of delay measurement

        Parameters:
            curve_name(str):the delay measurement you want to plot
            figsize(tuple):  figsize
            plt_show(bool):  if or not show and save figure.
            ax_external(object): external plot object
        """
        curve=getattr(self.delays, curve_name, None)
        if label is None:
            label = f"{self.data.meas_label}_{curve_name}"
        if curve is not None:
            plot_autocorr(
                curve, label, figsize=figsize, ax=ax_external, outlier=outlier
            )
            if plt_show:
                plt.tight_layout()
                plt.savefig(f"{PLOTS_DIR}{self.data.meas_label}/{curve_name}_autocorr_plot.png", dpi=300, bbox_inches='tight')
                plt.show()
        else:
            print(f"No attribute {curve_name} found in meas.delays of {self.data.meas_label}.")

    def plotAllAutoCorrelation(self, delays=None, labels = None, figsize=(24,24), subplot_division=[1,1], outlier=35, x_lim=100 ):
        if delays == None:
            curve_names = self.listDelaysAttr()
        else:
            curve_names = delays
        if labels == None:
            labels = curve_names
        figure_num = 0
        for i in range(0, len(curve_names)):
            if i % (subplot_division[0] * subplot_division[1]) == 0:
                if i != 0:
                    plt.tight_layout()
                    plt.savefig(
                        f"{PLOTS_DIR}{self.data.meas_label}/AutoCorrelation_{figure_num}.png",
                        dpi=300,
                    )
                    figure_num = figure_num + 1
                    plt.show()
                _, axs = plt.subplots(
                    subplot_division[0], subplot_division[1], figsize=figsize
                )

            if subplot_division[0] * subplot_division[1] == 1:
                self.plotAutoCorrelation(curve_names[i], label=labels[i],plt_show=False, ax_external=axs, outlier=outlier)
            elif subplot_division[0] == 1 or subplot_division[1] == 1:
                self.plotAutoCorrelation(
                    curve_names[i],
                    label=labels[i],
                    plt_show=False,
                    ax_external=axs[i % (subplot_division[1] * subplot_division[0])],
                    outlier=outlier,
                )
            else:
                self.plotAutoCorrelation(
                    curve_names[i],
                    label=labels[i],
                    plt_show=False,
                    ax_external=axs[
                        (
                            i
                            % (subplot_division[1] * subplot_division[0])
                            // subplot_division[1]
                        ),
                        i % subplot_division[1],
                    ],
                    outlier=outlier,
                )
        plt.tight_layout()
        plt.savefig(
            f"{PLOTS_DIR}{self.data.meas_label}/AutoCorrelation_{figure_num}.png",
            dpi=300,
            bbox_inches="tight",
        )
        plt.show()

    def crosscorr(self, datax, datay, lag=0, wrap=False):
        """Lag-N cross correlation with time-lagging capability.
        
        Parameters
        ----------
        datax, datay : pandas.Series
            The time series data to compute the cross-correlation.
        lag : int, default 0
            The lag (positive or negative) for the cross-correlation.
        wrap : bool, default False
            If True, wraps the lagged values (useful for cyclic data).
            
        Returns
        ----------
        crosscorr : float
            The Pearson correlation coefficient for the given lag.
        """
        if wrap:
            shiftedy = datay.copy()  # Create a copy to modify
            if lag > 0:
                # Wrap last `lag` values to the beginning
                shiftedy.iloc[:lag] = datay.iloc[-lag:].values
            elif lag < 0:
                # Wrap first `-lag` values to the end
                shiftedy.iloc[lag:] = datay.iloc[:lag + len(datay)].values
            # Compute correlation
            return datax.corr(shiftedy)
        else:
            # Standard shift with NaN filling
            return datax.corr(datay.shift(lag))

    def plotTLCC(self, curve_names, curve_labels=None, wrap = False):
        data = dict()
        if curve_labels is None:
            curve_labels = curve_names
        for idx, curve_name in enumerate(curve_names):
            curve = getattr(self.delays, curve_name, None)
            if curve is not None:
                data[curve_labels[idx]] = curve
            else:
                print(f"{curve_name} not found")
                return 

        # Create a pandas DataFrame with each array as a column
        df = pd.DataFrame(data)
        d1 = df[curve_labels[0]]
        d2 = df[curve_labels[1]]

        max_lag = 50  
        lags = range(-max_lag, max_lag + 1)

        rs = [self.crosscorr(d1, d2, lag, wrap=wrap) for lag in lags]

        peak_lag = lags[np.argmax(np.abs(rs))]
        d2_offset = -peak_lag

        f, ax = plt.subplots(figsize=(14, 3))
        ax.plot(lags, rs, label="Cross-correlation")
        ax.axvline(0, color='k', linestyle='--', label='Center (Lag=0)')
        ax.axvline(peak_lag, color='r', linestyle='--', label='Peak Synchrony')
        ax.set(
            title=(
                f"{curve_labels[0]}[i-{d2_offset}] ~ {curve_labels[1]}[i]"
                if d2_offset > 0
                else (
                    f"{curve_labels[0]}[i] ~ {curve_labels[1]}[i{d2_offset}]"
                    if d2_offset < 0
                    else f"{curve_labels[0]}[i] ~ {curve_labels[1]}[i]"
                )
            ),
            ylim=[-1, 1],
            xlabel="Lag (packets)",
            ylabel="Pearson r",
        )
        ax.legend()
        plt.show()
        return 

### 3.1 Usage of ```class Meas``` 

#### 3.1.1 Import dataset

In [281]:
if IF_SHOW_USAGE == True:
    Meas_s54=Meas(meas_label='s54')

#### Some test

In [282]:
#Meas_s49.data.sched_sorted_dict[Meas_s49.data.sched_sorted_dict.keys()[1123]]["cause"]["tbs"]
# Meas_s49.checkData("mcs_sorted_dict")
# Meas_s49.checkData("packets")

In [283]:
if IF_SHOW_USAGE== True:
    print(list(vars(Meas_s54.delays).keys()))  # ['attr1', 'attr2']

    for attr_name in list(vars(Meas_s54.delays).keys()):
        attr = getattr(Meas_s54.delays, attr_name, None)
        if attr is not None:
            print(
                f"len(Meas_{Meas_s54.data.meas_label}.delays.{attr_name})= {len(attr)} "
            )

In [284]:
if IF_SHOW_USAGE == True:
    print(type(Meas_s54.delays.tbss))
    print(Meas_s54.delays.tbss.shape)

#### 3.1.2 ```checkData```, ```listDataAttr```, ```listDelaysAttr```
```checkData```: print the [pkt_idx]-th json object in the given Meas.Data attribute name 

```listDataAttr```: print all attributes names in the given Meas.Data

```listDelaysAttr``` : print all attributes names in the given Meas.Delays

Here are all attributes retrieved from json files, stored in [Meas_instance].data

In [285]:
if IF_SHOW_USAGE == True:
    Meas_s54.listDataAttr()

In [286]:
if IF_SHOW_USAGE == True:
    Meas_s54.listDelaysAttr()

In [287]:
if IF_SHOW_USAGE == True:
    Meas_s54.checkData("sched_arr")

#### 3.1.3 ```plotCCDF``` plot CCDF of single delay component

In [288]:
if IF_SHOW_USAGE == True:
    print(list(vars(Meas_s54.delays).keys()))  # ['attr1', 'attr2']

In [289]:
if IF_SHOW_USAGE == True:
    Meas_s54.plotCCDF("scheduling_delays")

In [290]:
if IF_SHOW_USAGE == True:
    Meas_s54.plotCCDF("queueing_delays_wo_scheduling_delay")
    Meas_s54.plotCCDF("queueing_delays")

#### 3.1.4 ```plotAllCCDF``` plot all CCDF curves 

In [291]:
if IF_SHOW_USAGE == True:
    Meas_s54.plotAllCCDF(subplot_division=[5, 3])

# Meas_s49.plotAllCCDF(subplot_division=[1, 1])

# Meas_s49.plotAllCCDF(subplot_division=[1, 3])

# Meas_s49.plotAllCCDF(subplot_division=[3, 1])

#### 3.1.5 ```plotTimeSeries``` plot data values v.s. index

In [292]:
if IF_SHOW_USAGE == True:
    Meas_s54.plotTimeSeries(
        ["frame_alignment_delays", "scheduling_delays", "ran_delays"]
    )

In [293]:
if IF_SHOW_USAGE == True:
    Meas_s54.plotTimeSeries(
        ["RAN Delay w/o frame-alignment delay", "RAN Delay w/o scheduling delay"],
        curves=[
            Meas_s54.delays.ran_delays - Meas_s54.delays.frame_alignment_delays,
            Meas_s54.delays.ran_delays - Meas_s54.delays.scheduling_delays,
        ],
        marker="o",
    )

## 4 ```class MultiMeas``` for visualizing multiple datasets

In [294]:
class MultiMeas:
    def __init__(self, datasets_dir=DATASETS_DIR, meas_labels=["s39","s40","s49"]):
        """
        MultiMeas:
            save and visualize multiple Meas objects

        Parameters:
            datasets_dir(str): Path of all datasets;
            meas_label(list of str): The measurement label;
        """
        self.meas=[]
        self.meas_labels=meas_labels
        for meas_label in meas_labels:
            self.meas.append(Meas(datasets_dir=datasets_dir, meas_label=meas_label))
        self.folder_label="_".join(meas_labels)
        if not os.path.exists(PLOTS_DIR + self.folder_label):
            os.makedirs(PLOTS_DIR + self.folder_label)

    def plotCCDF(self, curve_names, meas_names, skip_first=SKIP_FIRST, skip_last=SKIP_LAST, figsize=(8,5), plt_show=True, ax_external=None, title='ccdf_plot'):
        """
        MultiMeas.plotCCDF:
            plot 1 ccdf among 13 kinds of delay measurement

        Parameters:
            curve_names(list of str):the delay measurement you want to plot
            meas_names(list of str):
            skip_first(int): skip first a few packets
            skip_last(int):  skip last a few packets
            figsize(tuple):  figsize
            plt_show(bool):  if or not show and save figure
            ax_external(object) ax object
            title(str): title (and filename)
        """
        if ax_external is None:
            _, ax=plt.subplots(figsize=figsize)
        else:
            ax=ax_external
        meas_list=[meas for meas in self.meas if meas.meas_label in meas_names]
        for one_meas in meas_list:
            for curve_name in curve_names:
                one_meas.plotCCDF(curve_name, skip_first=skip_first, skip_last=skip_last,ax_external=ax, plt_show=False)
        if plt_show:
            i = 1
            while os.path.exists(os.path.join(PLOTS_DIR, self.folder_label, f"{title}_{i}.png")):
                i += 1
            ax.set_title(f"{title}_{i}")
            plt.tight_layout()
            plt.savefig(
                f"{PLOTS_DIR}{self.folder_label}/{title}_{i}.png",
                dpi=300,
                bbox_inches="tight",
            )
            plt.show()

    def plotAllPerDelayType(self, figsize=(24,24),subplot_division=[1,1]):
        """
        Multieas.plotAllPerDelayType:
             plot CCDFs from multiple meas files on the same axes
        Parameters:
            figsize(tuple):  figsize
            subplot_division(list=[x,y]): x-row y-column subplots. If [1,1], figures will be plotted separately.
        """
        curve_names = self.meas[0].listDelaysAttr()
        curve_names = [
            curve_name for curve_name in curve_names if curve_name != "timestamps"
        ]
        figure_num=0
        for i in range(0,len(curve_names)):
            if i % (subplot_division[0]*subplot_division[1]) == 0:
                if i != 0:
                    plt.tight_layout()
                    plt.savefig(f"{PLOTS_DIR}{self.folder_label}/all_ccdf_plots_{figure_num}.png", dpi=300)
                    figure_num=figure_num+1
                    plt.show()
                _, axs = plt.subplots(subplot_division[0], subplot_division[1], figsize=figsize)

            if subplot_division[0]*subplot_division[1]==1:
                self.plotCCDF([curve_names[i]],self.meas_labels, plt_show=False, ax_external=axs)
            elif subplot_division[0]==1 or subplot_division[1]==1:
                self.plotCCDF([curve_names[i]],self.meas_labels, plt_show=False, ax_external=axs[i % (subplot_division[1]*subplot_division[0])])
            else:
                self.plotCCDF(
                    [curve_names[i]],
                    self.meas_labels,
                    plt_show=False,
                    ax_external=axs[
                        (
                            i
                            % (subplot_division[1] * subplot_division[0])
                            // subplot_division[1]
                        ),
                        i % subplot_division[1],
                    ],
                )
        plt.tight_layout()
        plt.savefig(f"{PLOTS_DIR}{self.folder_label}/all_ccdf_plots_{figure_num}.png", dpi=300, bbox_inches="tight")
        plt.show()

    def plotHistograms(self, delay_name, ax_external=None, skip_first=SKIP_FIRST, skip_last=SKIP_LAST, y_log=True, outlier=None, figsize=(8,5)):
        """
        Plots histograms for multiple arrays side by side on the same axes with normalized frequency.

        :param delay_name: The delay compoent to plot
        :param ax: Axes object to plot on
        :param skip_first: only take [skip_first:-skip_last] to plot
        :param skip_last:  only take [skip_first:-skip_last] to plot
        :param labels: List of labels for each array
        :param y_log: Boolean to set y-axis to log scale if True
        :param outlier: Cap value for outliers (optional)
        """

        values_per_meas = []
        labels = []
        for one_meas in self.meas:
            one_delay=getattr(one_meas.delays, delay_name, None)
            if one_delay is not None:
                values_per_meas.append(one_delay[skip_first:-skip_last])
                labels.append(one_meas.data.meas_label)
            else:
                print(f"No attribute {delay_name} found in meas.delays of {one_meas.data.meas_label}.")

        if ax_external is not None:
            ax=ax_external
        else:
            _, ax=plt.subplots(figsize=figsize)

        plot_multiple_histograms(values_per_meas=values_per_meas, ax=ax, labels=labels, y_log=y_log, delay_type_label=delay_name, outlier=outlier)
        plt.savefig(f"{PLOTS_DIR}{self.folder_label}/{delay_name}_hist_plot.png", dpi=300, bbox_inches="tight")
        plt.show()

    def plotTimeSeries(self, curve_names, meas_names, start_idx=2500, end_idx=2700, figsize=(12,5), marker="*", title="timeseries"):
        """
        Plot time series of multiple variables of multiple datasets in one plot.

        Parameters:
            curve_names(list of str): attribute names in Meas.Delays
            meas_names(list of str): dataset names
            start_idx(int): start index
            end_idx(int): end index
            figsize((int, int)): figsize
            marker(char): marker style of datapoints
        """
        _, ax = plt.subplots(figsize=figsize)

        meas_list=[meas for meas in self.meas if meas.meas_label in meas_names]
        for one_meas in meas_list:
            one_meas.plotTimeSeries(curve_names, start_idx=start_idx, end_idx=end_idx, figsize=figsize, marker=marker, ax_external=ax, plt_show=False)
        i = 0
        while os.path.exists(os.path.join(PLOTS_DIR, self.folder_label, f"{title}_{i}.png")):
            i += 1
        plt.tight_layout()
        plt.savefig(
            f"{PLOTS_DIR}{self.folder_label}/{title}_{i}.png", dpi=300, bbox_inches="tight"
        )
        plt.show()

    # def dataFrame(self, curve_names, curve_labels=None, csv_path=None, if_save=True):
    #     """
    #     Export dataframe of given delay components with given labels, and save it to csv file.

    #     curve_labels(list of str): data labels
    #     curve_names(list of str): attributes names of Meas.Delay (use Meas.listDelaysAttr() to check)
    #     csv_path(str): Optional. default: the data folder of the meas.
    #     """
        
    #     data = dict()
    #     for idx, curve_name in enumerate(curve_names):
    #         curve = getattr(self.delays, curve_name, None)
    #         if curve is not None:
    #             if curve_labels is not None:
    #                 data[curve_labels[idx]] = curve
    #             else:
    #                 data[curve_names[idx]] = curve

    #     # Create a pandas DataFrame with each array as a column
    #     df = pd.DataFrame(data)
    #     # Display the first few rows of the DataFrame
    #     print(df.head())

    #     # Export the DataFrame to a CSV file
    #     if csv_path == None:
    #         csv_path = f"{DATASETS_DIR}{self.data.meas_label}"
    #         print(csv_path)

    #     if if_save == True:
    #         df.to_csv(
    #             os.path.join(csv_path, f"{"_".join(curve_labels)}.csv"), index=True
    #         )
    #         print(
    #             f"Dataframe saved to {os.path.join(csv_path, f"{"_".join(curve_labels)}.csv")}"
    #         )

    #     return df

### 4.1 Usage of ```class MultiMeas```

#### 4.1.1 Import datasets

In [295]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54 = MultiMeas(meas_labels=["s39", "s40", "s54"])

In [296]:
if IF_SHOW_USAGE == True:
    print(Meas_s39_s40_s54.meas[0].data.meas_label)
    print(Meas_s39_s40_s54.meas_labels)

#### 4.1.2 check Data 

In [297]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[0].checkData("packets")

#### 4.1.3 ```plotCCDF``` plot CCDFs of certain delay components from certain datasets

In [298]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.plotCCDF(
        ["ran_delays", "scheduling_delays"], ["s39", "s49", "s54"]
    )

In [299]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.plotCCDF(
        ["queueing_delays_wo_scheduling_delay", "queueing_delays", "scheduling_delays"],
        ["s39"],
    )

#### 4.1.4 ```plotAllPerDelayType```  plot CCDFs from multiple meas files on the same axes

In [300]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.plotAllPerDelayType(subplot_division=[5, 3])

#### 4.1.5 ```plotHistograms``` 
Plots histograms for multiple arrays side by side on the same axes with normalized frequency.

In [301]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.plotHistograms("segments")

In [302]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.plotHistograms("mcss")

In [303]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.plotCCDF(["packet_size"], ["s39"])

#### 4.1.6 ```plotTimeSeries```


In [304]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.plotTimeSeries(
        [
            "segmentation_delay",
            "scheduling_delays",
            "segmentation_delays_wo_scheduling_delay",
            "segments",
        ],
        ["s39"],
        start_idx=2500,
        end_idx=2600,
    )

In [305]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.plotTimeSeries(
        [
            "queueing_delays",
            "scheduling_delays",
            "queueing_delays_wo_scheduling_delay",
        ],
        ["s39"],
        start_idx=2500,
        end_idx=2600,
    )

## 5 Export to DataFrame

In [306]:
if IF_SHOW_USAGE == True:
    df = Meas_s39_s40_s54.meas[0].dataFrame(
        [
            "tbss",
            "segments",
            "packet_size",
            "timestamps",
            "segmentation_delays_wo_scheduling_delay",
            "queueing_delays_wo_scheduling_delay",
        ],  # attrbutes name of Meas.delays
        [
            "TBS",
            "SegmentsNum",
            "PacketSize",
            "TimeStamps",
            "SegmentDelay(noSched)",
            "QueueingDelay(noSched)",
        ],  # labels to display (optional)
    )

## 6 Show Correlation

### 6.1 ```plotCrossCorrelation``` Correlation Coef Matrix

In [307]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[0].plotCrossCorrelation(
        ["mcss", "tbss", "segments", "packet_size"],  # attrbutes name of Meas.delays
        ["MCS", "TBS", "SegmentsNumber", "PacketSize"],  # labels to display (optional)
    )

In [308]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[1].plotAllAutoCorrelation(

        subplot_division=[8, 2],

    )


### 6.2 ```plotTLCC(d1, d2)``` 

**offset =-2**  $\Rightarrow$ ```d1[i]``` impacted by ```d2[i-2]```

**offset = 1** $\Rightarrow$ ```d1[i]``` impacted by ```d2[i+1]``` (wrong)
 $\Rightarrow$ ```d2[i]``` impacted by ```d1[i-1]``` (true)

In [309]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[0].plotTLCC(
        ["segments", "packet_size"],  # attrbutes name of Meas.delays
        ["SegmentsNumber", "PacketSize"],  # labels to display (optional)
    )

In [310]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[0].plotTLCC(

        ["segments", "tbss"],  # attrbutes name of Meas.delays

    )

# number of segments[i] ~ TBS [i-2]

In [311]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[0].plotTLCC(
        ["mcss", "segments"],  # attrbutes name of Meas.delays
        ["MCS", "SegmentsNumber"],  # labels to display (optional)
    )

In [312]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[0].plotTLCC(

        ["mcss", "tbss"],  # attrbutes name of Meas.delays

    )

In [313]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[1].plotTLCC(

        ["queueing_delays", "scheduling_delays"]  # attrbutes name of Meas.delays

    )

In [314]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[1].plotTLCC(
        [
            "queueing_delays_wo_scheduling_delay",
            "segmentation_delays_wo_scheduling_delay",
        ],  # attrbutes name of Meas.delays
    )

In [315]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[1].plotTLCC(
        [
            "segmentation_delays_wo_scheduling_delay",
            "frame_alignment_delays",
        ],  # attrbutes name of Meas.delays
    )

In [316]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[0].plotTLCC(

        [

            "mcss",

            "segments",

        ],  # attrbutes name of Meas.delays

    )

### Test

In [317]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40_s54.meas[0].listDataAttr()

### Show length


In [318]:
if IF_SHOW_USAGE == True:
    print(list(vars(Meas_s39_s40_s54.meas[0].delays).keys()))  # ['attr1', 'attr2']

    for attr_name in list(vars(Meas_s39_s40_s54.meas[0].delays).keys()):
        attr = getattr(Meas_s39_s40_s54.meas[0].delays, attr_name, None)
        if attr is not None:
            print(
                f"len(Meas_{Meas_s39_s40_s54.meas[0].data.meas_label}.delays.{attr_name})= {len(attr)} "
            )
# print(len(Meas_s39_s40_s49.meas[1].delays.queueing_delays))
# print(len(Meas_s39_s40_s49.meas[1].delays.frame_alignment_delays))
# print(len(Meas_s39_s40_s49.meas[1].delays.segmentation_delay))
# print(len(Meas_s39_s40_s49.meas[1].delays.mcss))

In [319]:
if IF_SHOW_USAGE == True:
    queueing_delay   = Meas_s39_s40_s54.meas[0].delays.queueing_delays
    scheduling_delay = Meas_s39_s40_s54.meas[0].delays.scheduling_delays

    diff = queueing_delay - scheduling_delay
    diff = diff[diff<0]
    len(diff)
    print(diff)

In [320]:
if IF_SHOW_USAGE == True:
    len(Meas_s39_s40_s54.meas[0].data.packets)

In [321]:
if IF_SHOW_USAGE == True:
    Meas_s39 = Meas(meas_label="s39")

In [322]:
if IF_SHOW_USAGE == True:
    print(Meas_s39.data.sched_sched_sorted_dict[1729237464.116453])

In [323]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40 = MultiMeas(meas_labels=["s39", "s40"])

In [324]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40.plotHistograms("packet_size", figsize=[20, 5])

In [325]:
if IF_SHOW_USAGE == True:
    Meas_s39_s40.plotHistograms("segments", figsize=[12, 5])

# 7 s59~s63

In [326]:
Meas_s39_40_s59s63 = MultiMeas(meas_labels=["s39","s40","s59", "s60", "s61", "s62", "s63"])

RNTIs in packets of s39: ['dc46']
2024-12-03 10:34:45.777 | ERROR    | decomp:get_tx_delay:246 - Packet 16138 phy.in_t or phy.in_t not present
2024-12-03 10:34:45.874 | ERROR    | decomp:get_tx_delay:246 - Packet 10013 phy.in_t or phy.in_t not present
2024-12-03 10:34:46.140 | ERROR    | decomp:get_tx_delay:246 - Packet 16138 phy.in_t or phy.in_t not present
2024-12-03 10:34:46.258 | ERROR    | decomp:get_tx_delay:246 - Packet 10013 phy.in_t or phy.in_t not present
2024-12-03 10:34:47.415 | ERROR    | decomp:get_tx_delay:246 - Packet 16138 phy.in_t or phy.in_t not present
2024-12-03 10:34:47.485 | ERROR    | decomp:get_tx_delay:246 - Packet 10013 phy.in_t or phy.in_t not present
2024-12-03 10:34:47.768 | ERROR    | decomp:get_tx_delay:246 - Packet 16138 phy.in_t or phy.in_t not present
2024-12-03 10:34:47.919 | ERROR    | decomp:get_tx_delay:246 - Packet 10013 phy.in_t or phy.in_t not present
2024-12-03 10:34:48.535 | ERROR    | decomp:get_tx_delay:246 - Packet 16138 phy.in_t or phy.in

In [327]:
print(list(vars(Meas_s39_40_s59s63.meas[3].delays).keys()))  # ['attr1', 'attr2']

for attr_name in list(vars(Meas_s39_40_s59s63.meas[3].delays).keys()):
    attr = getattr(Meas_s39_40_s59s63.meas[3].delays, attr_name, None)
    if attr is not None:
        print(
            f"len(Meas_{Meas_s39_40_s59s63.meas[3].data.meas_label}.delays.{attr_name})= {len(attr)} "
        )

['idt', 'frame_alignment_delays', 'scheduling_delays', 'ran_delays', 'ran_delays_wo_frame_alignment_delay', 'ran_delays_wo_scheduling_delay', 'queueing_delays', 'queueing_delays_wo_scheduling_delay', 'segmentation_delay', 'segmentation_delays_wo_scheduling_delay', 'segments', 'retx_delays', 'mcss', 'tbss', 'packet_size', 'timestamps']
len(Meas_s60.delays.idt)= 39985 
len(Meas_s60.delays.frame_alignment_delays)= 39984 
len(Meas_s60.delays.scheduling_delays)= 39984 
len(Meas_s60.delays.ran_delays)= 39986 
len(Meas_s60.delays.ran_delays_wo_frame_alignment_delay)= 28716 
len(Meas_s60.delays.ran_delays_wo_scheduling_delay)= 28716 
len(Meas_s60.delays.queueing_delays)= 39984 
len(Meas_s60.delays.queueing_delays_wo_scheduling_delay)= 39984 
len(Meas_s60.delays.segmentation_delay)= 39984 
len(Meas_s60.delays.segmentation_delays_wo_scheduling_delay)= 39984 
len(Meas_s60.delays.segments)= 39984 
len(Meas_s60.delays.retx_delays)= 39986 
len(Meas_s60.delays.mcss)= 39984 
len(Meas_s60.delays.tbss)=

In [ ]:
df = Meas_s39_40_s59s63.meas[6].dataFrame(
    [
        "tbss",
        "segments",
        "packet_size",
        "segmentation_delays_wo_scheduling_delay",
        "queueing_delays_wo_scheduling_delay",
    ],  # attrbutes name of Meas.delays
    [
        "TBS",
        "SegmentsNum",
        "PacketSize",
        "SegmentDelay(noSched)",
        "QueueingDelay(noSched)",
    ],  # labels to display (optional)
)

IndexError: list index out of range

In [329]:
jsons_path = "./data/s60/jsons/"
test_packets = read_json(os.path.join(jsons_path, "packets.json"))
test_segmentation_delay = np.array(
    list(
        {
            packet["id"]: get_segmentation_delay(packet)
            for packet in test_packets
            if get_segmentation_delay(packet) != None
        }.values()
    )
)

len(test_segmentation_delay)

2024-12-03 10:38:13.037 | ERROR    | decomp:get_tx_delay:246 - Packet 39478 phy.in_t or phy.in_t not present
2024-12-03 10:38:13.038 | ERROR    | decomp:get_tx_delay:246 - Packet 39478 phy.in_t or phy.in_t not present
2024-12-03 10:38:13.341 | ERROR    | decomp:get_tx_delay:246 - Packet 33959 phy.in_t or phy.in_t not present
2024-12-03 10:38:13.342 | ERROR    | decomp:get_tx_delay:246 - Packet 33959 phy.in_t or phy.in_t not present
For packet(id=33959), tx_delay == None!
2024-12-03 10:38:13.477 | ERROR    | decomp:get_tx_delay:246 - Packet 30954 phy.in_t or phy.in_t not present
2024-12-03 10:38:13.478 | ERROR    | decomp:get_tx_delay:246 - Packet 30954 phy.in_t or phy.in_t not present
2024-12-03 10:38:14.133 | ERROR    | decomp:get_tx_delay:246 - Packet 20884 phy.in_t or phy.in_t not present
2024-12-03 10:38:14.134 | ERROR    | decomp:get_tx_delay:246 - Packet 20884 phy.in_t or phy.in_t not present
2024-12-03 10:38:14.151 | ERROR    | decomp:get_tx_delay:246 - Packet 20537 phy.in_t or 

39984